# EECS 16A Spring 2021 Homework 12

In [ ]:
from __future__ import division
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import sys

## Mechanical Least Squares - Optional IPython code for plotting

In [ ]:
# use for plotting
a = np.array([2,4,6,8])
b = np.array([2,6,7,8])

### Part (a)

In [ ]:
# find a linear model of the form:   b = xa
# According to the least squares formula, we can calculate xhat

xhat = ## YOUR CODE HERE -- Use the least squares formula to compute the optimal x
print('xhat: '+str(xhat))
bhat = xhat*a

# Calculate squared error
se = np.dot(b-bhat,b-bhat)
print('the squared error is '+str(se))

In [ ]:
#HERE IS HOW TO PLOT POINTS
plt.plot(a,b,'ob') 
#HERE IS HOW TO PLOT A LINE
plt.plot(a,bhat,'-r')
#HERE IS HOW TO CHANGE THE LIMITS OF THE PLOTTING WINDOW
plt.xlim([0,9]); plt.ylim([0,9]);

### Part (b)

In [ ]:
# Copies of the data points
a2 = a.copy()
b2 = b.copy()
A2 = ## YOUR CODE HERE -- set up the augmented matrix A.
print('The augmented A matrix is')
print(A2)

xhat2 = ## YOUR CODE HERE -- Use the least squares formula to compute the optimal x vector
print('xhat:')
print(xhat2)
bhat2 = np.dot(A2,xhat2)

# The squared error is given by
se2 = np.dot(b2 - bhat2,b2-bhat2)
print('the squared error is '+str(se2))

In [ ]:
#HERE IS HOW TO PLOT POINTS
plt.plot(a2,b2,'ob') 
#HERE IS HOW TO PLOT A LINE
plt.plot(a2,bhat2,'-r')
#HERE IS HOW TO CHANGE THE LIMITS OF THE PLOTTING WINDOW
plt.xlim([0,9]); plt.ylim([0,9]);

## GPS Receivers

In [ ]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import sys

In [ ]:
## RUN THIS FUNCTION BEFORE YOU START THIS PROBLEM
## This function will generate the gold code associated with the satellite ID using linear shift registers
## The satellite_ID can be any integer between 1 and 24
def Gold_code_satellite(satellite_ID):
    codelength = 1023
    registerlength = 10
    
    # Defining the MLS for G1 generator
    register1 = -1*np.ones(registerlength)
    MLS1 = np.zeros(codelength)
    for i in range(codelength):
        MLS1[i] = register1[9]
        modulo = register1[2]*register1[9]
        register1 = np.roll(register1,1)
        register1[0] = modulo
    
    # Defining the MLS for G2 generator
    register2 = -1*np.ones(registerlength)
    MLS2 = np.zeros(codelength)
    for j in range(codelength):
        MLS2[j] = register2[9]
        modulo = register2[1]*register2[2]*register2[5]*register2[7]*register2[8]*register2[9]
        register2 = np.roll(register2,1)
        register2[0] = modulo
    
    delay = np.array([5,6,7,8,17,18,139,140,141,251,252,254,255,256,257,258,469,470,471,472,473,474,509,512,513,514,515,516,859,860,861,862])
    G1_out = MLS1;
    shamt = delay[satellite_ID - 1]
    G2_out = np.roll(MLS2,shamt)
    
    CA_code = G1_out * G2_out
    
    return CA_code

### Part (a)

In [ ]:
def array_correlation(array1,array2):
    """ This function should return two arrays or a matrix with one row corresponding to 
    the offset and other to the correlation value. array1 and array2 do not have to be
    arrays of equal length. array2 is the argument that is shifted (i.e. the signature of the statellite), 
    array1 is the argument that is stationary, (i.e the received signal).
    """
    ## Use np.correlate with "FULL". Check out the helper page for it 
    correlated_array = np.correlate(array1,array2,'full')
    length1 = len(array1)
    length2 = len(array2)
    min_ind = min(length1,length2)
    max_ind = max(length1,length2)
    indices = np.linspace(-min_ind + 1, max_ind - 1, min_ind + max_ind -1)
    return (indices, correlated_array)
    
# Plot the auto-correlation of satellite 10 with itself. 
array_10 = Gold_code_satellite(10)
(ind_10, self_10) = array_correlation(array_10,array_10)
plt.figure(figsize=(16,4))
plt.stem(ind_10,self_10)

### Part (b)

In [ ]:
array_13 = Gold_code_satellite(13)
(ind_10_13, cross_10_13) = array_correlation(#insert first agument, #insert second argument)
plt.figure(figsize=(16,4))
plt.stem(ind_10_13,cross_10_13)

### Part (c)

In [ ]:
## THIS IS A HELPER FUNCTION FOR PART C
def integernoise_generator(length_of_noise):
    noise_array = np.random.randint(2, size = length_of_noise)
    noise_array = 2 * noise_array - np.ones(size(noise_array))
    return noise_array

## PART C CODE HERE
intrand_signal = integernoise_generator(1023)
(ind_intrand,cross_intrand) = array_correlation(#insert first agument, #insert second argument)
plt.figure(figsize=(16,4))
plt.stem(ind_intrand,cross_intrand)

### Part (d)

In [ ]:
## THIS IS A HELPER FUNCTION FOR PART D
def gaussiannoise_generator(length_of_noise):
    noise_array = np.random.normal(0,1,length_of_noise)
    return noise_array

## PART D CODE HERE
gaussrand_signal = gaussiannoise_generator(1023)
(ind_gaussrand,cross_gaussrand) = array_correlation(#insert first agument, #insert second argument)
plt.figure(figsize=(16,4))
plt.stem(ind_gaussrand,cross_gaussrand)

### Part (e)
Hint: You can use a absolute value threshold of 800 for the cross-correlation to detect if a given satellite is present. 

Hint: np.argwhere may be useful for detecting satellites with peaks. You can use other functions/ use a manual process.

In [ ]:
## THIS IS A HELPER FUNCTION FOR PART E, F AND G

## This function returns a 1 if peak (greater than threshold or less than -threshold) is found else it returns a 0.
def find_peak(correlation,threshold):
    max_value = np.amax(correlation)
    min_value = np.amin(correlation)
    if max_value > threshold:
        ret_value = 1
    elif min_value < -1*threshold:
        ret_value = 1
    else:
        ret_value = 0
    return ret_value

In [ ]:
# Part E
## 'np.load' FUNCTION IS USED TO LOAD THE RECIEVED SIGNAL DATA
signal1 = np.load('data1.npy')
visible_sat_e = np.zeros(24) #this can be used to keep track of which satellites are visible
thresh = 800

# YOUR CODE HERE --- A skeleton that might be useful is provided, you may write this in any other way:
for sat_ID in np.arange(1,24):
    gold_code = Gold_code_satellite(sat_ID)
    #YOUR CODE HERE
    #find the correlation of the signal with the gold code of each satellite
    #then find if any of the correlations cross the threshold value using the find-peak function
    

### Part (f)
Hint: You can use a absolute value threshold of 800 for the cross-correlation to detect if a given satellite is present. 

Hint: np.argwhere may be useful for detecting satellites with peaks. You can use other functions/ use a manual process.

In [ ]:
## DATA2.NPY IS THE RECEIVED SIGNAL ARRAY
signal2 = np.load('data2.npy')
visible_sat_f = np.zeros(24) #this can be used to keep track of which satellites are visible
thresh = 800

# YOUR CODE HERE --- A skeleton that might be useful is provided, you may write this in any other way
#Use this code block to first find which satellite is present

for sat_ID in np.arange(1,24):
    #find the correlation of the signal with the gold code of each satellite
    #then find if any of the correlations cross the threshold value using the find-peak function
            

In [ ]:
# Using the satellite you found in the block above, now decode the message using the code below

#First create an array with the signature gold code of the satellite you found
#WRITE A LINE HERE TO FIND THE CROSS CORRELATION OF SIGNAL2 LOADED ABOVE WITH THE CODE OF THE SATELLITE YOU FOUND
plt.figure(figsize=(16,8))
plt.stem(sat3_data)

## Looking at the figure visually you should be able to decode the message



### Part (g)
Hint: You can use a absolute value threshold of 800 for the cross-correlation to detect if a given satellite is present. 

Hint: np.argwhere may be useful for detecting satellites with peaks. You can use other functions/ use a manual process.

In [ ]:
#OPTIONAL PART
## DATA3.NPY IS RECEIVED SIGNAL ARRAY
signal3 = np.load('data3.npy')
visible_sat_g = np.zeros(24) #this can be used to keep track of which satellites are visible
thresh = 800
# YOUR CODE HERE

##  Audio File Matching

This notebook continues the audio file matching problem. Be sure to have song.wav and clip.wav in the same directory as the notebook.

In this notebook, we will look at the problem of searching for a small audio clip inside a song.

The song "Mandelbrot Set" by Jonathan Coulton is licensed under <a href="http://creativecommons.org/licenses/by-nc/3.0/">CC BY-NC 3.0</a>

If you have trouble playing the audio file in IPython, try opening it in a different browser. I encountered problem with Safari but Chrome works for me.

### Setup

In [ ]:
import numpy as np
import wave
import matplotlib.pyplot as plt
import scipy.io.wavfile
import operator
from IPython.display import Audio
%matplotlib inline

given_file = 'song.wav'
target_file = 'clip.wav'
rate_given,  given_signal  = scipy.io.wavfile.read(given_file)
rate_target, target_signal = scipy.io.wavfile.read(target_file)
given_signal  = given_signal[:2000000].astype(float)
target_signal = target_signal.astype(float)
def play_clip(start, end, signal=given_signal):
    scipy.io.wavfile.write('temp.wav', rate_given, signal[start:end].astype(np.int16))
    return Audio(url='temp.wav', autoplay=False)

def run_comparison(target_signal, given_signal, idxs=None):
    # Run everything if not called with idxs set to something
    if idxs is None:
        idxs = [i for i in range(len(given_signal)-len(target_signal))]
    return idxs, [vector_compare(target_signal, given_signal[i:i+len(target_signal)])
                for i in idxs]

play_clip(0, len(given_signal))

#scipy.io.wavfile.write(target_file, rate_given, (-0.125*given_signal[1380000:1380000+70000]).astype(np.int16))

We will load the song into the variable `given_signal` and load the short clip into the variable `target_signal`. Your job is to finish code that will identify the short clip's location in the song. The clip we are trying to find will play after executing the following block.

In [ ]:
Audio(url=target_file, autoplay=False)

### Part (e)
Your task is to define the function 'vector_compare' and run the following code. Because the song has a lot of data, you should use the provided examples from the previous parts of the problem before running the later code. Do you results here make sense given your answers to previous parts of the problem?

In [ ]:
def vector_compare(desired_vec, test_vec):
    """This function compares two vectors, returning a number.
    The test vector with the highest return value is regarded as being closest to the desired vector."""
   #YOUR CODE HERE

print("Vector compare test examples:")
print(vector_compare(np.array([1,1,1]), np.array([1,1,1])))
print(vector_compare(np.array([1,1,1]), np.array([-1,-1,-1])))
print(vector_compare(np.array([1,1,1]), np.array([-1,1,-1])))

Run the following code that runs `vector_compare` on every subsequence in the song- it will probably take at least 5 minutes. How do you interpret this plot to find where the clip is in the song?

In [ ]:
import time

t0 = time.time()
idxs, song_compare = run_comparison(target_signal, given_signal)
t1 = time.time()
plt.plot(idxs, song_compare)
print ("That took %(time).2f minutes to run" % {'time':(t1-t0)/60.0} )

In the space below, write code that uses `song_compare` to print the index of `given_signal` where `target_signal` begins. Then, verify that your answer is correct by playing the song at that index using the `play_clip` function.

In [ ]:
#YOUR CODE HERE